In [3]:
#Importing required libraries
from bs4 import BeautifulSoup
import requests 
import sys 
import json
import tweepy
import time 
import logging
import random
from datetime import datetime, timedelta

logging.basicConfig(level = logging.INFO)
logger = logging.getLogger()

In [2]:
#Import the script that creates the Twitter API
from config import create_api
api = create_api()

In [1]:
#Like and retweets mentions
def fav_retweet(api):
    logger.info('Retrieving tweets...')
    mentions = api.mentions_timeline(tweet_mode = 'extended')
    for mention in reversed(mentions):
        if mention.in_reply_to_status_id is not None or mention.user.id == api.me().id:
            # This tweet is a reply or I'm its author so, ignore it
            return
        
        if not mention.favorited:
            # Mark it as Liked, since we have not done it yet
            try:
                mention.favorite()
                logger.info(f"Liked tweet by {mention.user.name}")
            except Exception as e:
                logger.error("Error on fav", exc_info=True)
                
        if not mention.retweeted:
            # Retweet, since we have not retweeted it yet
            try:
                mention.retweet()
                logger.info(f"Retweeted tweet by {mention.user.name}")
            except Exception as e:
                logger.error("Error on fav and retweet", exc_info=True)
                
        print(mentions)

In [4]:
#We can test the function by running it and interacting with the bot account to see if the bot picks up tweets mentioning the Hashtags:
while True:
    fav_retweet(api)
    logger.info("Waiting...")
    time.sleep(10)
    

INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...
INFO:root:Retrieving tweets...
INFO:root:Waiting...


KeyboardInterrupt: 

In [4]:
#Follow users who follow your Twitter account
def follow_followers(api):
    logger.info("Retrieving and following followers")
    for follower in tweepy.Cursor(api.followers).items():
        if not follower.following:
            try:
                follower.follow()
                logger.info(f"Following {follower.name}")
            except tweepy.error.TweepError:
                pass

In [6]:
follow_followers(api)

INFO:root:Retrieving and following followers


In [5]:
#Retweet any tweets with a certain hashtags in the text
def retweet_tweets_with_hashtag(api, need_hashtags):
    if type(need_hashtags) is list:
        search_query = f"{need_hashtags} -filter:retweets"
        tweets = api.search(q=search_query, lang ="en", tweet_mode='extended')
        for tweet in tweets:
            hashtags = [i['text'].lower() for i in tweet.__dict__['entities']['hashtags']]
            try:
                need_hashtags = [hashtag.strip('#') for hashtag in need_hashtags]
                need_hashtags = list(need_hashtags)
                if set(hashtags) & set(need_hashtags):
                    if tweet.user.id != api.me().id:
                        api.retweet(tweet.id)
                        logger.info(f"Retweeted tweet from {tweet.user.name}")
                        time.sleep(5)
            except tweepy.TweepError:
                logger.error("Error on retweet", exc_info=True)
    else:
        logger.error("Hashtag search terms needs to be of type list", exc_info=True) 
        return

In [8]:
while True:
        retweet_tweets_with_hashtag(api, ["#manim"])
        logger.info("Waiting...")
        time.sleep(10)

ERROR:root:Error on retweet
Traceback (most recent call last):
  File "<ipython-input-7-1741744a048c>", line 13, in retweet_tweets_with_hashtag
    api.retweet(tweet.id)
  File "c:\users\jh231\appdata\local\programs\python\python38-32\lib\site-packages\tweepy\binder.py", line 253, in _call
    return method.execute()
  File "c:\users\jh231\appdata\local\programs\python\python38-32\lib\site-packages\tweepy\binder.py", line 234, in execute
    raise TweepError(error_msg, resp, api_code=api_error_code)
tweepy.error.TweepError: Read-only application cannot POST.
INFO:root:Waiting...
ERROR:root:Error on retweet
Traceback (most recent call last):
  File "<ipython-input-7-1741744a048c>", line 13, in retweet_tweets_with_hashtag
    api.retweet(tweet.id)
  File "c:\users\jh231\appdata\local\programs\python\python38-32\lib\site-packages\tweepy\binder.py", line 253, in _call
    return method.execute()
  File "c:\users\jh231\appdata\local\programs\python\python38-32\lib\site-packages\tweepy\binde

KeyboardInterrupt: 

In [6]:
#Creating a bot for continuous monitoring
def main():
    tweets = ["To deny people their Human Rights is to challange their very Humanity", "The idea of cultural realitivism is nothing but an excuse to violate Human Rights", "Injustice anywhere is a threat to Justice evrerywhere", "I am tired"]
    api = create_api()
    last_tweeted = datetime.now()-timedelta(hours=24)
    while True:
        fav_retweet_hashtag(api, "#whatshappeninginmyanmar", "#myanmar", "#civildisobediencemovement", "#r2p", "#milkteaalliancemyanmar")
        last_tweeted = tweet_daily(api, last_tweeted, random.choice(tweets))
        logger.info("Waiting...")
        time.sleep(60)